In [ ]:
import geopandas as gpd
#https://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
from osgeo import osr, gdal

def get_poly_with_lat_long_from_geotif(geotif_fp):
    # Import necessary modules


    # get the existing coordinate system
    ds = gdal.Open(geotif_fp)

    old_cs= osr.SpatialReference()

    old_cs.ImportFromWkt(ds.GetProjectionRef())

    # create the new coordinate system
    nad83_wkt = """
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]]"""

    new_cs = osr.SpatialReference()

    new_cs .ImportFromWkt(nad83_wkt)

    # create a transform object to convert between coordinate systems
    transform = osr.CoordinateTransformation(old_cs,new_cs) 

    #get the point to transform, pixel (0,0) in this case
    width = ds.RasterXSize
    height = ds.RasterYSize
    gt = ds.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5] 
    maxx = gt[0] + width*gt[1] + height*gt[2]
    maxy = gt[3]

    #get the coordinates in lat long
    latlong1 = transform.TransformPoint(minx,miny)
    #print(latlong1)
    latlong2 = transform.TransformPoint(minx,maxy)
    #print(latlong2)
    latlong3 = transform.TransformPoint(maxx,maxy)
    #print(latlong3)
    latlong4 = transform.TransformPoint(maxx,miny)
    #print(latlong4)
    
    # Create an empty geopandas GeoDataFrame
    polydata = gpd.GeoDataFrame()
    polydata['geometry'] = None

    # Create a Polygon
    #print(latlong1[0], latlong1[1]), (latlong2[0], latlong2[1]), (latlong3[0], latlong3[1]), (latlong4[0], latlong4[1])
    coords = [(latlong1[0], latlong1[1]), (latlong2[0], latlong2[1]), (latlong3[0], latlong3[1]), (latlong4[0], latlong4[1])]
    poly = Polygon(coords)
    #polydata.loc[0, 'geometry'] = poly
    #polydata.loc[0, 'id'] = 0
    #print(polydata)
    #newdata = newdata.append(poly, ignore_index=True)
    # Determine the output path for the Shapefile
    #outfp = r"/storage/images/charcoal_hearth_hill/images/slope/atestof_just_this_geotiff_border.shp"

    # Write the data into that Shapefile
    #polydata.to_file(outfp)
    return(poly)

import numpy as np

def tif_to_jpg(tif_file,output_file_name, output_folder):
    #print(tif_file,output_file_name, output_folder)
    ds = gdal.Open(tif_file)
    #geoTrans = srcImage.GetGeoTransform()

    band = ds.GetRasterBand(1)
    width = ds.RasterXSize
    height = ds.RasterYSize

    data = band.ReadAsArray(0, 0, width, height)
    #convert all the bad data
    data[data==-9999.0] = 0
    max_value = numpy.max(data)
    color_multiplier = 255/(max_value-1)
    #print(color_multiplier)
    data = data*color_multiplier
    #print(data)
    data_int = np.array(data, dtype='int')
    #print(data_int)
    #clip = ds.readasarray(ds)
    data_int = data_int.astype(gdalnumeric.numpy.uint8)

    gdalnumeric.SaveArray(data_int, output_folder+output_file_name+".jpg", format="JPEG")

#https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html?highlight=rasterize

# Run this
def write_annot(obj_type, obj_annots_dir, obj_f_num, org_f_path,org_f_width,org_f_height,org_f_depth, refPts):
    # With credit to: 
    # https://www.geeksforgeeks.org/reading-writing-text-files-python/
    annot_file_path = obj_annots_dir+obj_f_num+'.xml'
    annot_file = open(annot_file_path,"w") 

    annot_file.write("<annotation>\n") 
    annot_file.write("	<folder>"+obj_type+"</folder>\n") 
    annot_file.write("	<filename>"+obj_f_num+".jpg</filename>\n") 
    annot_file.write("	<path>"+org_f_path+"</path>\n") 
    annot_file.write("	<source>\n") 
    annot_file.write("		<database>Muhlenberg_charcoal_hearths</database>\n") 
    annot_file.write("	</source>\n") 
    annot_file.write("	<size>\n") 
    annot_file.write("		<width>"+str(org_f_width)+"</width>\n") 
    annot_file.write("		<height>"+str(org_f_height)+"</height>\n") 
    annot_file.write("		<depth>"+str(org_f_depth)+"</depth>\n") 
    annot_file.write("	</size>\n") 
    annot_file.write("	<segmented>0</segmented>\n") 
    for ocn in range(0,len(refPts)):
        refPt = refPts[ocn]
        refPtMin = refPt[0]
        refPtMax = refPt[1]
        
        #Sometimes the mouse gets dragged from the bottom to the top, etc
        if(refPtMin[1]>refPtMax[1]):
            ytemp = refPtMin[1]
            refPtMin[1] = refPtMax[1]
            refPtMax[1]=ytemp
        if(refPtMin[0]>refPtMax[0]):
            xtemp = refPtMin[0]
            refPtMin[0] = refPtMax[0]
            refPtMax[0]=xtemp 
        
        annot_file.write("	<object>\n") 
        annot_file.write("		<name>"+obj_type+"</name>\n") 
        annot_file.write("		<number>"+str(ocn)+"</number>\n") 
        annot_file.write("		<truncated>0</truncated>\n") 
        annot_file.write("		<difficult>0</difficult>\n") 
        annot_file.write("		<bndbox>\n") 
        annot_file.write("			<xmin>"+str(refPtMin[0])+"</xmin>\n") 
        annot_file.write("			<ymin>"+str(refPtMin[1])+"</ymin>\n") 
        annot_file.write("			<xmax>"+str(refPtMax[0])+"</xmax>\n") 
        annot_file.write("			<ymax>"+str(refPtMax[1])+"</ymax>\n") 
        annot_file.write("		</bndbox>\n") 
        annot_file.write("	</object>\n") 
    annot_file.write("</annotation>\n") 
    annot_file.close()

In [2]:
import gdal
from osgeo import ogr
def get_pixel_with_geot(geoT, point_x, point_y):
    """Get the pixel for given coordinates (in the raster's convention, not
    checked!) for a raster file.

    Parameters
    ----------
    raster: string
        A GDAL-friendly raster filename
    point_x: float
        The Easting in the same coordinates as the raster (not checked!)
    point_y: float
        The Northing in the same coordinates as the raster (not checked!)

    Returns
    -------
    The row/column (or column/row, depending on how you define it)
    """
    #g = gdal.Open(raster)
    #if g is None:
    #    raise ValueError(f"{raster:s} cannot be opened!")
    #geoT = open_raster.GetGeoTransform()
    inv_geoT = gdal.InvGeoTransform(geoT)
    r, c = (gdal.ApplyGeoTransform(inv_geoT, point_x, point_y))
    return int(r + 0.5), int(c + 0.5)

def annotate_tif_if_it_has_hearths(area_hearth_data, tif_poly, tif_fp, annot_file_name, geotif_crs):
    # Create an empty geopandas GeoDataFrame
    hearth_data = gpd.GeoDataFrame()
    hearth_data['geometry'] = None
    hearth_data.crs = {'init':'epsg:4326'}

    # Create an empty geopandas GeoDataFrame
    hearth_data_poly = gpd.GeoDataFrame()
    hearth_data_poly['geometry'] = None
    hearth_data_poly.crs = {'init':'epsg:4326'}

    selection = area_hearth_data[0:]
    point_count=0
    
    print(tif_fp)
    src_ds=gdal.Open(tif_fp) 
    gt=src_ds.GetGeoTransform()
    
    # create a transform object to convert between coordinate systems
    #transform = osr.CoordinateTransformation(old_cs,new_cs)
    
    target = osr.SpatialReference(wkt=src_ds.GetProjection())
    source = osr.SpatialReference()
    #source.ImportFromEPSG(4326)
    #source.ImportFromEPSG(32128)
    #target.ImportFromEPSG(32128)
    source.ImportFromEPSG(geotif_crs)
    target.ImportFromEPSG(geotif_crs)
    
    transform = osr.CoordinateTransformation(source, target)

    for index, row in selection.iterrows():
        #print("for index, row in selection.iterrows():")
        pt = row['geometry']
        #print(row)
        #select only points inside the bounds of the geotiff AND are confirmed to be hearths
        if(pt.within(tif_poly)==True):
            #print("if(pt.within(tif_poly)==True):")
            point_count=point_count+1
            #print(pt, pt.within(tif_poly))
            hearth_data = hearth_data.append(row, ignore_index=True)
            
            polygon_half_size = 0.0000895078
            polygon_half_size = 0.0001745402
            #get the coordinates in lat long
            # pp_ = point_poly - the polygon around the points
            pp_minx=pt.bounds[0]-polygon_half_size
            #print(pt.bounds[0])
            pp_maxx=pt.bounds[0]+polygon_half_size
            pp_miny=pt.bounds[1]-polygon_half_size
            pp_maxy=pt.bounds[1]+polygon_half_size
            
            pp_latlong1 = transform.TransformPoint(pp_minx,pp_miny)
            #print("latlong1",latlong1)
            print("pp_minx",pp_minx)
            pp_latlong2 = transform.TransformPoint(pp_minx,pp_maxy)
            #print(latlong2)
            pp_latlong3 = transform.TransformPoint(pp_maxx,pp_maxy)
            #print(latlong3)
            pp_latlong4 = transform.TransformPoint(pp_maxx,pp_miny)
            #print(latlong4)
            # Create a Polygon
            pp_coords = [(pp_minx, pp_miny), (pp_minx, pp_maxy), (pp_maxx, pp_maxy), (pp_maxx, pp_miny)]
            ppoly = Polygon(pp_coords)
            new_pp_row = {'id':row['id'], 'geometry':ppoly}
            #hearth_data_poly.loc[0, 'geometry'] = ppoly
            #hearth_data_poly.loc[0, 'id'] = row['id']
            hearth_data_poly = hearth_data_poly.append(new_pp_row, ignore_index=True)
    #Do we have points inside this tif?  If so, annotate and copy>0
    if(point_count)>0:

        #hearth_data_poly = hearth_data_poly.to_crs(epsg=32128)
        #hearth_data = hearth_data.to_crs(epsg=32128)
        hearth_data_poly = hearth_data_poly.to_crs(epsg=geotif_crs)
        hearth_data = hearth_data.to_crs(epsg=geotif_crs)
    
        #Write out shp files for diagnostics
        outfp_base = tif_fp[:-4]
        outfp = outfp_base+"_hearth_data_poly.shp"
        # Write the data into that Shapefile
        hearth_data_poly.to_file(outfp)
        outfp = outfp_base+"_hearth_data.shp"
        # Write the data into that Shapefile
        hearth_data.to_file(outfp)
        print("****")

        #Get the dimentions of column and row
        nc   = src_ds.RasterXSize
        nr   = src_ds.RasterYSize
        print("src_ds.RasterXSize", nc,nr)
        gdal.UseExceptions() #so it doesn't print to screen everytime point is outside grid
        
        #target = osr.SpatialReference(wkt=src_ds.GetProjection())
        #source = osr.SpatialReference()
        #source.ImportFromEPSG(4326)
        #transform = osr.CoordinateTransformation(source, target)
        refPts = []      
        refPt = []
        #I'm hoping the tifs will be 1024X768        
        # Make annotation file for the jpeg from polygons
        for index, row in hearth_data_poly.iterrows():
            poly = row['geometry']
            #print(row, row['id'])
            #print(poly)
            
            boundary = poly.bounds
            print("boundary = poly.bounds")
            print(boundary[0])
            print(boundary[1])
            print(boundary[2])
            print(boundary[3])

            lon = boundary[0]
            lat = boundary[1]
    
            print("lon,lat",lon, lat)
      
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(lon, lat)
            point.Transform(transform)

            #p1x,p1y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
            p1x,p1y = get_pixel_with_geot(gt, point.GetX(), point.GetY())
            print("p1",point.GetX(), point.GetY(),p1x,p1y)
        
            #cx,cy=convert_coordinates(lon, lat)
            #p1x,p1y= (get_pixel_with_geot(gt,cx,cy ))
            lon = boundary[2]
            lat = boundary[3]
    
            print("lon,lat",lon, lat)
        
            #cx,cy=convert_coordinates(lon, lat)
            #p2x,p2y= (get_pixel_with_geot(gt,cx,cy ))
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(lon, lat)
            point.Transform(transform)

            #p2x,p2y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
            p2x,p2y = get_pixel_with_geot(gt, point.GetX(), point.GetY())
            print("p2",point.GetX(), point.GetY(),p2x,p2y)

            #p2y is smaller than p1y so we'll exchange them to keep p1 as the minimum.
            ptemp=p1y
            p1y=p2y
            p2y=ptemp
            if(p1x<0):
                p1x=0
            if(p1y<0):
                p1y=0
            if(p2x<0):
                p2x=0
            if(p2y<0):
                p2y=0

            if(p1x>nc):
                p1x=nc
            if(p1y>nr):
                p1y=nr
            if(p2x>nc):
                p2x=nc
            if(p2y>nr):
                p2y=nr
     
        
            refPt = [(p1x, p1y)]
            refPt.append((p2x, p2y))
            refPts.append(refPt)

        object_type = 'charcoal_hearth_hill'
        object_annots_dir = '/storage/images/'+object_type+'/annots/'
        write_annot(object_type, object_annots_dir, annot_file_name, annot_file_name+".jpg",nc,nr,1, refPts)
        tif_to_jpg(tif_fp,annot_file_name,"/storage/images/charcoal_hearth_hill/images_training/")
    

In [3]:
# Thanks to
#https://gis.stackexchange.com/questions/92207/split-a-large-geotiff-into-smaller-regions-with-python-and-gdal
import os  
import numpy
#import gdal
from osgeo import gdal, osr
import math
from itertools import chain
#import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import gdalnumeric

def get_extent(dataset):

    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    transform = dataset.GetGeoTransform()
    minx = transform[0]
    maxx = transform[0] + cols * transform[1] + rows * transform[2]

    miny = transform[3] + cols * transform[4] + rows * transform[5]
    maxy = transform[3]

    return {
            "minX": str(minx), "maxX": str(maxx),
            "minY": str(miny), "maxY": str(maxy),
            "cols": str(cols), "rows": str(rows)
            }

def create_tiles(minx, miny, maxx, maxy, nw, nh):
    width = maxx - minx
    height = maxy - miny

    matrix = []

    for j in range(nw, 0, -1):
        for i in range(0, nh):

            ulx = minx + (width/nw) * i # 10/5 * 1
            uly = miny + (height/nh) * j # 10/5 * 1

            lrx = minx + (width/nw) * (i + 1)
            lry = miny + (height/nh) * (j - 1)
            matrix.append([[ulx, uly], [lrx, lry]])

    return matrix



def create_standard_tiles(minx, miny, maxx, maxy):
    print("standard",minx, miny, maxx, maxy)
    width = maxx - minx
    height = maxy - miny
    
    standard_width = 1024
    standard_height = 768
   
    nw = int(width/standard_width)+1
    nh = int(height/standard_height)+1
    print(nw,nh)
    #print(nw*standard_width,nh*standard_height)
    
    matrix = []

    for j in range(nh, 0, -1):
        for i in range(0, nw):

            ulx = minx + (standard_width * i) # 10/5 * 1
            uly = miny + (standard_height * j) # 10/5 * 1

            lrx = minx + (standard_width * (i + 1))
            lry = miny + (standard_height * (j - 1))
            if(lrx>maxx):
                lrx=maxx
            if(lrx<minx):
                lrx=minx
              
            if(lry>maxy):
                lry=maxy
            if(lry<miny):
                lry=miny

            if(ulx>maxx):
                ulx=maxx
            if(ulx<minx):
                ulx=minx

            if(uly>maxy):
                uly=maxy
            if(uly<miny):
                uly=miny

            #print("matrix:",ulx, uly,lrx, lry)
            matrix.append([[ulx, uly], [lrx, lry],[i,j]])
    print("standard",minx, miny, maxx, maxy)
    return matrix


def split(file_name, geotif_crs):
    raw_file_name = os.path.splitext(os.path.basename(file_name))[0].replace("_downsample", "")
    driver = gdal.GetDriverByName('GTiff')
    dataset = gdal.Open(file_name)
    proj_dataset = osr.SpatialReference(wkt=dataset.GetProjection())
    print(proj_dataset.GetAttrValue('AUTHORITY',1))
    print(proj_dataset)
    
    
    band = dataset.GetRasterBand(1)
    transform = dataset.GetGeoTransform()


    extent = get_extent(dataset)

    cols = int(extent["cols"])
    rows = int(extent["rows"])

    print ("Columns: ", cols)
    print ("Rows: ", rows)

    minx = float(extent["minX"])
    maxx = float(extent["maxX"])
    miny = float(extent["minY"])
    maxy = float(extent["maxY"])
    
    width = int(maxx - minx)
    height = int(maxy - miny)

    nw = int(width/1024)+1
    nh = int(height/768)+1
    
    #******
    # make a polygon for the area and see what points are in the whole area.
    #get the coordinates in lat long

    # Create an empty geopandas GeoDataFrame
    area_hearth_data = gpd.GeoDataFrame()
    area_hearth_data['geometry'] = None
    area_poly=get_poly_with_lat_long_from_geotif(file_name)
    # Read file using gpd.read_file()
    all_hearth_data = gpd.read_file("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp")
    selection = all_hearth_data[0:]
    #print(selection)
    for index, row in selection.iterrows():
        #print("for index, row in selection.iterrows():")
        pt = row['geometry']
        #print(row)
        #select only points inside the bounds of the geotiff AND are confirmed to be hearths
        if(pt.within(area_poly)==True):
            #print("if(pt.within(tif_poly)==True):")
            area_hearth_data = area_hearth_data.append(row, ignore_index=True)
    
    # Determine the output path for the Shapefile
    outfp = r"area_"+str(raw_file_name[:2])+"_hearth.shp"
    # Write the data into that Shapefile
    area_hearth_data.to_file(outfp)
    #*************************8
    output_path = os.path.join("data", raw_file_name)
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    #print ("GCD", math.gcd(round(width, 0), round(height, 0)))
    #print ("Width", width)
    #print ("height", height)

    # Polygons to reflect the tiles
    # Create an empty geopandas GeoDataFrame
    tile_boundary_poly = gpd.GeoDataFrame()
    tile_boundary_poly['geometry'] = None
    tile_boundary_poly.crs = {'init':'epsg:4326'}

    #tiles = create_tiles(minx, miny, maxx, maxy, nw,nh)
    tiles = create_standard_tiles(minx, miny, maxx, maxy)
    transform = dataset.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    srs = osr.SpatialReference()
    #srs.ImportFromEPSG(32128)
    srs.ImportFromEPSG(geotif_crs)
    #srs.ImportFromEPSG(4326)
    #dst_ds.SetProjection( srs.ExportToWkt() )
    
    tile_num = 0
    for tile in tiles:
        #print (tile)

        minx = tile[0][0]
        maxx = tile[1][0]
        miny = tile[1][1]
        maxy = tile[0][1]
        tilex = "00"+str(tile[2][0])
        tilex = tilex[-2:]
        tiley = "00"+str(tile[2][1])
        tiley = tiley[-2:]
        
        p1 = (minx, maxy)
        p2 = (maxx, miny)

        i1 = int((p1[0] - xOrigin) / pixelWidth)
        j1 = int((yOrigin - p1[1])  / pixelHeight)
        i2 = int((p2[0] - xOrigin) / pixelWidth)
        j2 = int((yOrigin - p2[1]) / pixelHeight)

        #print (i1, j1)
        #print (i2, j2)

        new_cols = i2-i1
        new_rows = j2-j1

        data = band.ReadAsArray(i1, j1, new_cols, new_rows)
        #print("data",data.mean(),data)
        #if len(list(set(chain(*data))))<3:
        if(np.all(data==data[0]) or (np.amin(data)==-9999 and np.amax(data)==0)):
            print("np.all(data==data[0])")
            #print("data.mean",data.mean())
        else:
            print("data.mean.too",data.mean(),np.amin(data),np.amax(data) )
            
            new_x = xOrigin + i1*pixelWidth
            new_y = yOrigin - j1*pixelHeight

            #print (new_x, new_y)

            new_transform = (new_x, transform[1], transform[2], new_y, transform[4], transform[5])
            output_file_name = raw_file_name[:2] + tiley + tilex
            output_file_base = output_file_name + ".tif"
            #output_file = os.path.join("/storage/images/charcoal_hearth_hill/images/split_tifs", raw_file_name, output_file_base)
            #output_file = os.path.join("/storage/images/charcoal_hearth_hill/images/split_tifs", raw_file_name, output_file_base)
            output_file = "/storage/images/charcoal_hearth_hill/images/split_tifs/"+output_file_base

            dst_ds = driver.Create(output_file,
                                   new_cols,
                                   new_rows,
                                   1,
                                   gdal.GDT_Float32)

            #writing output raster
            dst_ds.GetRasterBand(1).WriteArray( data )
            dst_ds.SetProjection( srs.ExportToWkt() )
            dst_ds.SetGeoTransform(new_transform)
            tif_metadata = {
                "minX": str(minx), "maxX": str(maxx),
                "minY": str(miny), "maxY": str(maxy)
            }
            dst_ds.SetMetadata(tif_metadata)
            #setting extension of output raster
            # top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
            #dst_ds.SetGeoTransform(new_transform)

            #wkt = dataset.GetProjection()

            # setting spatial reference of output raster
            #srs = osr.SpatialReference()
            #srs.ImportFromWkt(wkt)
            #dst_ds.SetProjection( srs.ExportToWkt() )

            dst_ds = None            

            #make a polygon for this tile
            tpoly = get_poly_with_lat_long_from_geotif(output_file)
            new_tp_row = {'id':tile_num, 'geometry':tpoly}
            tile_boundary_poly = tile_boundary_poly.append(new_tp_row, ignore_index=True)
            
            #Save .jpg
            # close tif before re-opening it and saving as jpg
            # The jpg is to be consumed by the model.
            tif_to_jpg(output_file,output_file_name,"/storage/images/charcoal_hearth_hill/images/jpgs/")
            #/storage/images/charcoal_hearth_hill/images_training/
            #Are there points inside the geotiff?
            #annotate_tif_if_it_has_hearths("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp",tpoly,output_file,output_file_name)
            annotate_tif_if_it_has_hearths(area_hearth_data,tpoly,output_file,output_file_name,geotif_crs)
            
            #We need to refine to handle points on the border
            
        #Close output raster dataset
        dst_ds = None

        tile_num += 1

    dataset = None
    
    # Determine the output path for the Shapefile
    tile_boundary_poly = tile_boundary_poly.to_crs(epsg=geotif_crs)
    
    outfp = "/storage/images/charcoal_hearth_hill/polys/"+raw_file_name[:2]+"_tile_boundary.shp"
    # Write the data into that Shapefile
    tile_boundary_poly.to_file(outfp)
    #print(tile_boundary_poly)


In [ ]:
split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/39/39blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/41/41blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/43/43blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/46/46blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/52/52partialDEM_Slope.tif",26918)
split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/55/55blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/58/58blast2demPAN_Slope_DEM.tif",32128)
#